In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.externals import joblib
from sklearn import cross_validation
from sklearn import grid_search

from sklearn import ensemble
from sklearn import linear_model

In [5]:
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'linear'

In [6]:
training_features = pd.read_csv('data/training_features.csv')
test_features = pd.read_csv('data/test_features.csv')

In [7]:
X_full = training_features.ix[:,2:]
y_full = training_features.SeriousDlqin2yrs

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_full, y_full, test_size=0.2, stratify=y_full, random_state=42)

In [9]:
'''clf = joblib.load('models/rfc-2017-01-07-20-54-40.pkl')'''

clf = linear_model.LogisticRegression(n_jobs=-1, random_state=42)
clf.fit(X_train_full, y_train_full)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
flename = 'models/%s-%s.pkl' % (model_name, timestamp)

joblib.dump(clf, flename)

log = '%s,%s\n' % (flename, clf)

fle = open('model_log.csv', 'a')
fle.write(log)
fle.flush()
fle.close()

print flename

models/linear-2017-01-08-20-26-50.pkl


In [11]:
X_submission = test_features.ix[:,2:]
y_submission = clf.predict_proba(X_submission.astype(np.float32))

In [12]:
submission = pd.DataFrame()
submission['Id'] = test_features.ID.astype(np.int)
submission['Probability'] = y_submission[:,1]
submission.head()

,Id,Probability
0,1,0.055430
1,2,0.064107
2,3,0.012073
3,4,0.106529
4,5,0.069087


In [13]:
flename = 'submissions/submission-%s-%s.csv' % (model_name, timestamp)
submission.to_csv(flename, index=False)
print flename

submissions/submission-linear-2017-01-08-20-26-50.csv
